# Стохастический градиентный и координатный спуски (практика)

In [187]:
# Загрузим требуемые библиотеки
import numpy as np
import pandas as pd
import sklearn
import sympy
#from sympy import Symbol, S, diff, solveset, Eq
from sympy import *


In [188]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
#from sklearn.metrics import mean_squared_error

In [189]:
# Загрузим исходный датасет
df = pd.read_csv('data\Advertising.csv',index_col=0)
df.head()

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [190]:
# проверим наличие пропусков
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


*Пропуски в данных отсутствуют*

In [191]:
# # Получим долю пропусков в каждом столбце тренировочной выборки.
# cols_null_percent = df.isnull().mean() * 100
# cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
# display(cols_with_null)

In [192]:
# выделим предикторы и целевую переменную
X = df[['TV','radio','newspaper']]
y = df['sales']

### Координатный спуск

In [193]:
# Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
y = y.values.reshape(-1, 1)
print(X.shape, y.shape)

(200, 4) (200, 1)


In [194]:
# Нормализуем данные
X = X / np.sqrt(np.sum(np.square(X), axis=0))

In [195]:
# Реализуем алгоритм координатного спуска:
# num_iters = количество итераций
# m = количество строк в матрице X
# n = количество столбцов в матрице X
# w = вектор размера nx1, состояющий из нулей
num_iters = 1000
n =X.shape[1]
m = X.shape[0]
w = np.zeros((n,1))

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # Вычисляем функцию потерь
        cost = sum(((X @ w) - y) ** 2)/(len(y))
w

array([[ 41.56217205],
       [110.13144155],
       [ 73.52860638],
       [ -0.55006384]])

In [196]:
# Сравним результаты с реализацией линейной регрессии из библиотеки sklearn:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
print(model.coef_)

[[ 41.56217205 110.13144155  73.52860638  -0.55006384]]


### Стохастический градиентный спуск

In [197]:
import numpy as np
from sklearn import preprocessing
minmax_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

# Отмасштабируем  столбцы исходной матрицы X:


df = pd.read_csv('data\Advertising.csv',index_col=0)
X = df[['TV','radio','newspaper']]
y = df['sales']

#X = minmax_scaler.fit_transform(X)
X=(X-np.mean(X, axis=0))/np.std(X, axis=0)

X
# X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
# y = y.values.reshape(-1, 1)
# X = minmax_scaler.fit_transform(X)
# means = np.mean(X, axis=0)
# stds = np.std(X, axis=0)
# X2 = (X - means) / stds
# X2

,TV,radio,newspaper
1,0.969852,0.981522,1.778945
2,-1.197376,1.082808,0.669579
3,-1.516155,1.528463,1.783549
4,0.052050,1.217855,1.286405
5,0.394182,-0.841614,1.281802
...,...,...,...
196,-1.270941,-1.321031,-0.771217
197,-0.617035,-1.240003,-1.033598
198,0.349810,-0.942899,-1.111852
199,1.594565,1.265121,1.640850


In [198]:
y

1      22.1
2      10.4
3       9.3
4      18.5
5      12.9
       ... 
196     7.6
197     9.7
198    12.8
199    25.5
200    13.4
Name: sales, Length: 200, dtype: float64

In [199]:
X

,TV,radio,newspaper
1,0.969852,0.981522,1.778945
2,-1.197376,1.082808,0.669579
3,-1.516155,1.528463,1.783549
4,0.052050,1.217855,1.286405
5,0.394182,-0.841614,1.281802
...,...,...,...
196,-1.270941,-1.321031,-0.771217
197,-0.617035,-1.240003,-1.033598
198,0.349810,-0.942899,-1.111852
199,1.594565,1.265121,1.640850


In [200]:
# Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
# y = np.array(y).reshape(-1, 1)
X

array([[ 1.00000000e+00,  9.69852266e-01,  9.81522472e-01,
         1.77894547e+00],
       [ 1.00000000e+00, -1.19737623e+00,  1.08280781e+00,
         6.69578760e-01],
       [ 1.00000000e+00, -1.51615499e+00,  1.52846331e+00,
         1.78354865e+00],
       [ 1.00000000e+00,  5.20496822e-02,  1.21785493e+00,
         1.28640506e+00],
       [ 1.00000000e+00,  3.94182198e-01, -8.41613655e-01,
         1.28180188e+00],
       [ 1.00000000e+00, -1.61540845e+00,  1.73103399e+00,
         2.04592999e+00],
       [ 1.00000000e+00, -1.04557682e+00,  6.43904671e-01,
        -3.24708413e-01],
       [ 1.00000000e+00, -3.13436589e-01, -2.47406325e-01,
        -8.72486994e-01],
       [ 1.00000000e+00, -1.61657614e+00, -1.42906863e+00,
        -1.36042422e+00],
       [ 1.00000000e+00,  6.16042873e-01, -1.39530685e+00,
        -4.30581584e-01],
       [ 1.00000000e+00, -9.45155670e-01, -1.17923146e+00,
        -2.92486143e-01],
       [ 1.00000000e+00,  7.90028350e-01,  4.96973404e-02,
      

In [201]:
# Создадим функцию mse_error для вычисления среднеквадратичной ошибки
def mse_error (A, B):
    mse = ((A - B)**2).mean(axis=0)
    return mse

In [202]:
# Сделаем наивный прогноз: предскажем продажи средним значением
x_m = df['sales'].mean()
y_pred = df['sales'] = df['sales'].apply(lambda x: x_m)
y_pred

1      14.0225
2      14.0225
3      14.0225
4      14.0225
5      14.0225
        ...   
196    14.0225
197    14.0225
198    14.0225
199    14.0225
200    14.0225
Name: sales, Length: 200, dtype: float64

In [203]:
# рассчитаем среднеквадратичную ошибку для этого прогноза:
mse_error (y, y_pred )

27.085743750000002

In [204]:
# проверим полученный трезультат для среднеквадратичной ошибки с помощью библиотеки sklearn:
from sklearn.metrics import mean_squared_error
mean_squared_error(y, y_pred)

27.085743750000002

In [205]:
# true_value_of_y = [3,2,6,1,5]
# predicted_value_of_y = [2.0,2.4,2.8,3.2,3.6]
# summation_of_value = 0 
# n = len(true_value_of_y) 
# for i in range (0,n):
#   difference_of_value = true_value_of_y[i] - predicted_value_of_y[i]  
#   squared_difference = difference_of_value**2  
#   summation_of_value = summation_of_value + squared_difference
# MSE = summation_of_value/n  
# print ("The Mean Squared Error is: " , MSE)

In [206]:
# from sklearn.metrics import mean_squared_error
# y_true = [2,-1,0.6,-0.7,-0.2]
# y_pred = [0.9,0.1,-0.4,-0.1,0.6]
# mean_squared_error(y_true, y_pred)

In [207]:
# Создадим функцию, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов
def lin_pred (X, w):
    y_pred = np.dot(X, w)
    return y_pred

In [208]:
y

1      22.1
2      10.4
3       9.3
4      18.5
5      12.9
       ... 
196     7.6
197     9.7
198    12.8
199    25.5
200    13.4
Name: sales, Length: 200, dtype: float64

In [212]:
lin_pred (X, w)

array([[ 2.19564846e+02],
       [-1.10575596e+01],
       [-1.40094509e+01],
       [ 1.36134050e+02],
       [ 2.23862737e+01],
       [-1.01899642e+01],
       [-2.60646867e+01],
       [-1.06685700e+01],
       [-2.40802793e+02],
       [ 7.04974099e+00],
       [-1.49075544e+02],
       [ 1.32895668e+02],
       [-5.90568819e+01],
       [-9.93281236e+01],
       [ 1.62388430e+02],
       [ 2.24506310e+02],
       [ 3.75619537e+00],
       [ 2.94812341e+02],
       [-7.19553548e+01],
       [ 4.53410199e+01],
       [ 1.54773186e+02],
       [ 6.77570627e+01],
       [-1.67602002e+02],
       [ 1.14572153e+02],
       [-1.20051510e+02],
       [ 9.27072631e+01],
       [ 6.66577725e+01],
       [ 1.28837426e+02],
       [ 1.91660649e+02],
       [-9.30667865e+01],
       [ 2.53816759e+02],
       [-3.16627680e+01],
       [-1.30577198e+02],
       [ 1.78586661e+02],
       [-1.32430435e+02],
       [ 1.31715296e+02],
       [ 2.98304380e+02],
       [ 7.79098116e+01],
       [-7.5

In [210]:
mean_squared_error(y, lin_pred(X, w))

17836.972606146024